In [ ]:

!wget -q https://github.com/DM-GY-9103-2024F-H/9103-utils/raw/main/src/data_utils.py
!wget -q https://github.com/DM-GY-9103-2024F-H/9103-utils/raw/main/src/image_utils.py
!wget -q https://github.com/DM-GY-9103-2024F-H/WK13/raw/main/final_utils.py

!wget -qO- https://github.com/DM-GY-9103-2024F-H/9103-utils/releases/latest/download/lfw.tar.gz | tar xz

Data Processing

In [ ]:
import os
import random

random.seed(42)

artist_names = [
    "Albrecht_Dürer",
    "Alfred_Sisley",
    "Amedeo_Modigliani",
    "Andy_Warhol",
    "Diego_Velazquez",
    "Edgar_Degas",
    "Francisco_Goya",
    "Frida_Kahlo",
    "Giotto_di_Bondone",
    "Gustav_Klimt",
    "Henri_Matisse",
    "Hieronymus_Bosch",
    "Joan_Miro",
    "Kazimir_Malevich",
    "Leonardo_da_Vinci",
    "Marc_Chagall",
    "Mikhail_Vrubel",
    "Pablo_Picasso",
    "Paul_Gauguin",
    "Paul_Klee",
    "Peter_Paul_Rubens",
    "Pierre_Auguste_Renoir",
    "Pieter_Bruegel",
    "Raphael",
    "Rembrandt",
    "Rene_Magritte",
    "Salvador_Dali",
    "Sandro_Botticelli",
    "Titian",
    "Vincent_van_Gogh"
]

artists_dict = {index: name for index, name in enumerate(artist_names, start=1)}

file_names = [f"file{i}" for i in range(1, 102)]

random.shuffle(file_names)

train_files = file_names[:81]
test_files = file_names[81:]

